## Version 1 chatgpt structure

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

# Step 1: Data Cleaning
def clean_data(signal, threshold):
    # Apply artifact rejection based on thresholding
    cleaned_signal = np.copy(signal)
    cleaned_signal[np.abs(signal) > threshold] = np.nan
    return cleaned_signal

# Step 2: Resampling
def resample_data(signal, original_fs, target_fs):
    resampled_signal = signal[::int(original_fs / target_fs)]
    return resampled_signal, target_fs

# Step 3: Referencing
def apply_reference(signal, reference_type):
    if reference_type == 'CAR':
        mean_signal = np.nanmean(signal, axis=1, keepdims=True)
        referenced_signal = signal - mean_signal
    elif reference_type == 'bipolar':
        referenced_signal = np.diff(signal, axis=0)
    else:
        referenced_signal = signal  # No referencing
    return referenced_signal

# Step 4: Filtering
def apply_bandpass_filter(signal, fs, lowcut, highcut):
    b, a = signal.butter(4, [lowcut, highcut], btype='bandpass', fs=fs)
    filtered_signal = signal.lfilter(b, a, signal)
    return filtered_signal

# Step 5: Segmentation
def segment_data(signal, segment_length):
    num_segments = int(len(signal) / segment_length)
    segments = np.array_split(signal[:num_segments * segment_length], num_segments)
    return segments

# Step 6: Artifact Removal (optional)
def remove_artifacts(signal, artifact_indices):
    cleaned_signal = np.copy(signal)
    cleaned_signal[artifact_indices] = np.nan
    return cleaned_signal

# Step 7: Normalization (optional)
def normalize_data(signal):
    normalized_signal = (signal - np.nanmean(signal)) / np.nanstd(signal)
    return normalized_signal

# Step 8: Feature Extraction (example)
def extract_features(signal):
    features = np.mean(signal, axis=1)  # Example: Compute the mean of each segment
    return features

# Example usage:
# Load iEEG data
data = np.loadtxt('ieeg_data.txt')

# Step 1: Data Cleaning
threshold = 100  # Adjust threshold based on the data characteristics
cleaned_data = clean_data(data, threshold)

# Visualize the cleaned data
plt.plot(cleaned_data)
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.title('Cleaned Data')
plt.show()

# Step 2: Resampling
original_fs = 1000  # Original sampling frequency in Hz
target_fs = 250  # Target sampling frequency in Hz
resampled_data, fs = resample_data(cleaned_data, original_fs, target_fs)

# Visualize the resampled data
plt.plot(resampled_data)
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.title('Resampled Data')
plt.show()

# Step 3: Referencing
reference_type = 'CAR'  # Choose reference type: 'CAR', 'bipolar', or 'none'
referenced_data = apply_reference(resampled_data, reference_type)

# Visualize the referenced data
plt.plot(referenced_data)
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.title('Referenced Data')
plt.show()

# Step 4: Filtering
lowcut = 1  # Low-pass filter cutoff frequency in Hz
highcut = 100  # High-pass filter cutoff frequency in Hz
filtered_data = apply_bandpass_filter(referenced_data, fs, lowcut, highcut)

# Visualize the filtered data
plt.plot(filtered_data)
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.title('Filtered Data')
plt.show()

# Step 5: Segmentation
segment_length = 5 * fs  # Segment length in seconds (example: 5 seconds)
segments = segment_data(filtered_data, segment_length)

# Visualize the segmented data
for segment in segments:
    plt.plot(segment)
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.title('Segmented Data')
plt.show()

# Step 6: Artifact Removal (optional)
# artifact_indices = detect_artifacts(filtered_data)  # Identify artifact indices
# cleaned_data = remove_artifacts(filtered_data, artifact_indices)

# Step 7: Normalization (optional)
# normalized_data = normalize_data(cleaned_data)

# Step 8: Feature Extraction
features = extract_features(segments)

# Display the results
plt.plot(features)
plt.xlabel('Segment')
plt.ylabel('Feature Value')
plt.title('Extracted Features')
plt.show()


OSError: ieeg_data.txt not found.

## version 2 chatgpt code


In [ ]:
# LOAD PARAMETERS

# TODO: Response plots -2 to 2 seconds
# Check preferences for parallelizing for speed (cap on memory)
import os
import numpy as np
import matplotlib.pyplot as plt
import mne
from mne.time_frequency import tfr_morlet, tfr_multitaper

data_dir = 'S:/HumanNeuronLab/DATARAW/PAT_3975'
data_file = 'PAT_3975_EEG_1631437_FLM_anon.TRC'
trig_file_imag = 'PAT_3975__FLM_picture__triggerPD_2023-04-27.tsv'
trig_file_read = ''
trig_file_audi = ''
events_file = 'sub-PAT_3975_task-LanguageMapping_timestamp-30-3-2023(10h44m14s)_lang-GER_events.tsv'
patient_number = '3975'

# PREPROCESSING

# read events file
try:
    evts = mne.read_events(os.path.join(data_dir, events_file), verbose=False)
except FileNotFoundError:
    evts = mne.read_events(os.path.join(data_dir, events_file), verbose=False, skiprows=4)

# read iEEG file header and trigger information
raw = mne.io.read_raw(os.path.join(data_dir, data_file), preload=True)

# Read stimuli presented (onset) and response (offset) values from the photodiode file.
trig_imag = np.loadtxt(os.path.join(data_dir, trig_file_imag), skiprows=1)
trig_audi = np.loadtxt(os.path.join(data_dir, trig_file_audi), skiprows=1)
trig_read = np.loadtxt(os.path.join(data_dir, trig_file_read), skiprows=1)
# Combine all three categories
trig = np.concatenate((np.column_stack((trig_imag, np.ones_like(trig_imag[:, 0]))),
                       np.column_stack((trig_audi, np.ones_like(trig_audi[:, 0]) * 2)),
                       np.column_stack((trig_read, np.ones_like(trig_read[:, 0]) * 3))))
nTrl = trig.shape[0]

# load iEEG data in memory with minimal preprocessing
ch_names = raw.info['ch_names']
cfg_preproc = dict(dataset=os.path.join(data_dir, data_file),
                   tmin=trig[:, 2] - raw.info['sfreq'],
                   tmax=trig[:, 3] + (3 * raw.info['sfreq']),
                   baseline=(-0.25, 0),
                   detrend=1,
                   picks=np.arange(len(ch_names)),
                   reject=None,
                   preload=True)
data = mne.io.read_raw_egi(**cfg_preproc)

# define parameters for TF analysis
cfg_timefreq = dict(output='power', method='multitaper', tmin=-0.5, tmax=2.0, fmin=2.0, fmax=200.0, n_jobs=1)
freqs = np.arange(2., 201., 2.)
n_cycles = freqs / 2.0
cfg_timefreq.update(dict(freqs=freqs, n_cycles=n_cycles))

# perform TF analysis
tfr_epochs = tfr_multitaper(data, **cfg_timefreq)

# perform TF baselining
tfr_epochs.apply_baseline((-0.25, 0), mode='percent')

# define parameters for TF data selection
cfg_TFselect = dict(average=True)

# PLOT STIM-LOCKED TFs, from 0.5 second before to


## Plot per channel

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import mne

# define parameters for TF baselining
cfg_TFbl = dict()
cfg_TFbl['baseline'] = [-0.25, 0]
cfg_TFbl['baselinetype'] = 'db'
cfg_TFbl['parameter'] = 'powspctrm'

# define parameters for TF data selection
cfg_TFselect = dict()
cfg_TFselect['avgoverrpt'] = 'yes'

# define parameters for TF plotting
cfg_TFplot = dict()
cfg_TFplot['zlim'] = 'maxabs'
cfg_TFplot['interactive'] = 'no'
cfg_TFplot['figure'] = 'no'
cfg_TFplot['colormap'] = 'jet'

catLabel = ['picture naming', 'auditory definition', 'reading completion']

for chan in range(len(data.ch_names)):
    
    # perform TF analysis
    cfg_timefreq = dict()
    cfg_timefreq['channel'] = data.ch_names[chan]
    freq = mne.time_frequency.tfr_morlet(data, **cfg_timefreq)
    
    # perform TF baselining
    freqBL = mne.time_frequency.tfr_baseline(freq, **cfg_TFbl)

    # PLOT STIM-LOCKED TFs, from 0.5 second before to 2 seconds after stimulus onset
    for ctTask in range(3):

        # select trials per task (WITH averaging)
        cfg_TFselect['trials'] = trig[:, 4] == ctTask + 1
        freq_plot = mne.time_frequency.tfr_morlet(freqBL, **cfg_TFselect)

        # do the actual plotting
        plt.subplot(3, 2, ctTask*2+1)
        cfg_TFplot['figure'] = plt.gcf()
        cfg_TFplot['title'] = [freq_plot.ch_names[0] + ' stim: ' + catLabel[ctTask]]
        cfg_TFplot['xlim'] = [-0.5, 2]
        mne.viz.plot_tfr_single_trial(freq_plot, **cfg_TFplot)

        # draw lines for baseline period and stimulus onset
        plt.axvline(-0.25, linestyle='--')
        plt.axvline(0)

    # PLOT RESPONSE-LOCKED TFs, from 1 second before to 2 seconds after response onset
    for ctTask in range(3):

        # this part realigns individual trials, based on the detection of
        # the first NaN (which automatically occurs 3 seconds after the
        # "response" trigger), which allows to go around the variable
        # trial duration

        # keep only trials of the current task
        freq_tmp = freqBL
        freq_tmp.data = freq_tmp.data[trig[:, 4] == ctTask + 1, :, :, :]

        # start building re-aligned freq structure
        freq_realign = freq_tmp.copy()
        freq_realign.data = np.empty((freq_tmp.data.shape[0], freq_tmp.data.shape[1], freq_tmp.data.shape[2], len(np.arange(-1, 2.01, 0.01))))
        freq_realign.times = np.arange(-1, 2.01, 0.01)

        # do the actual realignment
        for ctTrl in range(freq_tmp.data.shape[0]):
            # find index of the time point just before the first NaN; this corresponds to 2.75 seconds after response onset
            idx_last = np.where
